In [1]:
import os
import cv2
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [2]:
import os
from zipfile import ZipFile


!pip install kaggle


os.environ['KAGGLE_USERNAME'] = "majisouvik1099"
os.environ['KAGGLE_KEY'] = "9464225218d56f8bfea9dd9cc437489f"


!kaggle datasets download -d jessicali9530/lfw-dataset


with ZipFile('lfw-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('lfw-dataset')


os.remove('lfw-dataset.zip')


 84% 95.0M/112M [00:00<00:00, 184MB/s]
100% 112M/112M [00:00<00:00, 190MB/s] 


In [3]:

def extract_features(image_path, model):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = image.unsqueeze(0)
    with torch.no_grad():
        features = model(image)
    features = features.squeeze(0)
    return features


def compute_hog(img):
    resized_img = resize(img, (128*4, 64*4))
    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, multichannel=True)
    return fd


def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))
    val_ar.append(get_pixel(img, center, x, y+1))
    val_ar.append(get_pixel(img, center, x+1, y+1))
    val_ar.append(get_pixel(img, center, x+1, y))
    val_ar.append(get_pixel(img, center, x+1, y-1))
    val_ar.append(get_pixel(img, center, x, y-1))
    val_ar.append(get_pixel(img, center, x-1, y-1))
    val_ar.append(get_pixel(img, center, x-1, y))

    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val

def calcLBP(img):
    height, width, channel = img.shape
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_lbp = np.zeros((height, width,3), np.uint8)
    for i in range(0, height):
        for j in range(0, width):
             img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
    hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
    return hist_lbp.flatten()


In [4]:
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 127MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [5]:
lfw_folder = '/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'
X, y = [], []
for folder_name in os.listdir(lfw_folder):
    folder_path = os.path.join(lfw_folder, folder_name)
    if os.path.isdir(folder_path):
        num_images = len(os.listdir(folder_path))
        if num_images > 70:
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_name)
                image = imread(image_path)
                # hog_feature = compute_hog(image)
                # lbp_feature = calcLBP(image)
                cnn_feature = extract_features(image_path, resnet).numpy()
                # hog_feature = hog_feature.reshape(-1)
                # lbp_feature = lbp_feature.reshape(-1)
                cnn_feature = cnn_feature.flatten()
                # combined_feature = np.concatenate((hog_feature, lbp_feature, cnn_feature))

                X.append(cnn_feature)
                y.append(folder_name)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [9]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
parameters = [
        {"kernel":
            ["linear"],
            "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000]},
        {"kernel":
            ["poly"],
            "degree": [2, 3, 4],
            "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000]},
        {"kernel":
            ["rbf"],
            "gamma": ["auto", "scale"],
            "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000]}
]

svm_clf = SVC()
print("tuning hyperparameters via grid search")
grid = GridSearchCV(estimator=SVC(), param_grid=parameters, n_jobs=-1)
grid.fit(X_train,y_train_encoded)
print(f"grid search best score: {grid.best_score_ * 100:.2f}%")
print(f"grid search best parameters: {grid.best_params_}")



tuning hyperparameters via grid search


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


grid search best score: 79.22%
grid search best parameters: {'C': 10, 'degree': 2, 'kernel': 'poly'}


In [12]:
model = SVC(C= 10, degree= 2, kernel= 'poly')
model.fit(X_train,y_train_encoded)
y_pred_svm = model.predict(X_test)
# print(classification_report(y_test_encoded, y_pred_svm))
print("SVM Classifier")
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.93      0.93      0.93        14
     Colin_Powell       0.83      0.85      0.84        40
  Donald_Rumsfeld       0.84      0.76      0.80        34
    George_W_Bush       0.81      0.88      0.84       109
Gerhard_Schroeder       0.60      0.52      0.56        23
      Hugo_Chavez       0.50      0.45      0.48        11
       Tony_Blair       0.78      0.67      0.72        27

         accuracy                           0.79       258
        macro avg       0.76      0.72      0.74       258
     weighted avg       0.79      0.79      0.79       258



In [13]:
y_pred_svm = model.predict(X_train)
accuracy_svm = accuracy_score(y_train_encoded, y_pred_svm)
print("SVM Classifier")
print("Training Accuracy:", accuracy_svm)

SVM Classifier
Training Accuracy: 0.9990291262135922


In [14]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
model=SVC()
model.fit(X_train_pca, y_train_encoded)
y_pred = model.predict(X_test_pca)
accuracy = accuracy_score(y_test_encoded, y_pred)
print("SVM Classifier")
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_))

SVM Classifier
Accuracy: 0.7248062015503876
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       1.00      0.64      0.78        14
     Colin_Powell       0.77      0.85      0.81        40
  Donald_Rumsfeld       1.00      0.44      0.61        34
    George_W_Bush       0.66      0.96      0.78       109
Gerhard_Schroeder       0.75      0.26      0.39        23
      Hugo_Chavez       0.83      0.45      0.59        11
       Tony_Blair       0.81      0.48      0.60        27

         accuracy                           0.72       258
        macro avg       0.83      0.58      0.65       258
     weighted avg       0.77      0.72      0.70       258



In [15]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
parameters = [
        {"kernel":
            ["linear"],
            "C": [0.1, 1, 10, 100,1000]},
        {"kernel":
            ["poly"],
            "degree": [2, 3, 4],
            "C": [0.1, 1, 10, 100,1000]},
        {"kernel":
            ["rbf"],
            "gamma": ["auto", "scale"],
            "C": [0.1, 1, 10, 100,1000]}
]

svm_clf = SVC()
print("tuning hyperparameters via grid search")
grid = GridSearchCV(estimator=SVC(), param_grid=parameters, n_jobs=-1)
grid.fit(X_train_pca,y_train_encoded)
print(f"grid search best score: {grid.best_score_ * 100:.2f}%")
print(f"grid search best parameters: {grid.best_params_}")



tuning hyperparameters via grid search


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


grid search best score: 77.09%
grid search best parameters: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}


In [18]:
model = SVC(C= 10, gamma= 'auto', kernel= 'rbf')
model.fit(X_train_pca,y_train_encoded)
y_pred_svm = model.predict(X_test_pca)
# print(classification_report(y_test_encoded, y_pred_svm))
print("SVM Classifier")
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.92      0.86      0.89        14
     Colin_Powell       0.81      0.88      0.84        40
  Donald_Rumsfeld       0.92      0.71      0.80        34
    George_W_Bush       0.81      0.92      0.86       109
Gerhard_Schroeder       0.59      0.57      0.58        23
      Hugo_Chavez       0.67      0.55      0.60        11
       Tony_Blair       0.82      0.67      0.73        27

         accuracy                           0.81       258
        macro avg       0.79      0.73      0.76       258
     weighted avg       0.81      0.81      0.80       258



In [20]:
y_pred_svm = model.predict(X_train_pca)
accuracy_svm = accuracy_score(y_train_encoded, y_pred_svm)
print("SVM Classifier")
print("Accuracy:", accuracy_svm)

SVM Classifier
Accuracy: 1.0


In [21]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components=None)
X_train_lda = lda.fit_transform(X_train, y_train_encoded)
X_test_lda = lda.transform(X_test)

model.fit(X_train_lda, y_train_encoded)
y_pred_svm = model.predict(X_test_lda)
accuracy_svm = accuracy_score(y_test_encoded, y_pred_svm)
print("SVM Classifier")
print("Accuracy:", accuracy_svm)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Accuracy: 0.5891472868217055
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       1.00      0.21      0.35        14
     Colin_Powell       0.95      0.50      0.66        40
  Donald_Rumsfeld       0.89      0.24      0.37        34
    George_W_Bush       0.52      0.95      0.68       109
Gerhard_Schroeder       0.46      0.26      0.33        23
      Hugo_Chavez       0.00      0.00      0.00        11
       Tony_Blair       0.85      0.41      0.55        27

         accuracy                           0.59       258
        macro avg       0.67      0.37      0.42       258
     weighted avg       0.67      0.59      0.54       258



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
parameters = [
        {"kernel":
            ["linear"],
            "C": [0.1, 1, 10, 100,1000]},
        {"kernel":
            ["poly"],
            "degree": [2, 3, 4],
            "C": [0.1, 1, 10, 100,1000]},
        {"kernel":
            ["rbf"],
            "gamma": ["auto", "scale"],
            "C": [0.1, 1, 10, 100,1000]}
]

svm_clf = SVC()
print("tuning hyperparameters via grid search")
grid = GridSearchCV(estimator=SVC(), param_grid=parameters, n_jobs=-1)
grid.fit(X_train_lda,y_train_encoded)
print(f"grid search best score: {grid.best_score_ * 100:.2f}%")
print(f"grid search best parameters: {grid.best_params_}")



tuning hyperparameters via grid search


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


grid search best score: 99.81%
grid search best parameters: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}


In [23]:
model = SVC(C= 0.1, gamma= 'scale', kernel= 'rbf')
model.fit(X_train_lda,y_train_encoded)
y_pred_svm = model.predict(X_test_lda)
# print(classification_report(y_test_encoded, y_pred_svm))
print("SVM Classifier")
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.86      0.86      0.86        14
     Colin_Powell       0.84      0.80      0.82        40
  Donald_Rumsfeld       0.86      0.74      0.79        34
    George_W_Bush       0.83      0.90      0.86       109
Gerhard_Schroeder       0.47      0.65      0.55        23
      Hugo_Chavez       0.60      0.27      0.37        11
       Tony_Blair       0.91      0.74      0.82        27

         accuracy                           0.79       258
        macro avg       0.77      0.71      0.72       258
     weighted avg       0.80      0.79      0.79       258



In [24]:
y_pred_svm = model.predict(X_train_lda)
accuracy_svm = accuracy_score(y_train_encoded, y_pred_svm)
print("SVM Classifier")
print("Accuracy:", accuracy_svm)

SVM Classifier
Accuracy: 0.9990291262135922
